### Quality Control
This notebook examines the output netCDF files, and does some basic quality control checks to make sure values in the netCDFs match their sources in the CSV files.

In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import random
from pathlib import Path
from luts import *

# source file directory
stats_dir = Path('/beegfs/CMIP6/jdpaul3/hydroviz_data/stats')
# output directory
nc_dir = Path('/beegfs/CMIP6/jdpaul3/hydroviz_data/maurer/nc_stats_fix')

# get stats list
stats = list(stat_vars_dict.keys())


In [5]:
# first read stats CSVs and do some filtering of results ...
# this is similar to the code in filter_files() in functions.py, but runs independently here
files = stats_dir.glob('*.csv')
seg_files = []
hru_files = []
for f in files:
    if "Maurer" in f.name: pass
    elif "diff" in f.name: pass
    elif "1952_2005" in f.name: pass
    elif "hru" in f.name:
        hru_files.append(f)
    else: 
        seg_files.append(f)

In [6]:
# load netCDFs
seg = xr.open_dataset(os.path.join(nc_dir, "seg.nc"))
hru = xr.open_dataset(os.path.join(nc_dir, "hru.nc"))

In [7]:
# check out the datasets - structure should be identical except for length of stream_ids
seg

<xarray.Dataset> Size: 7GB
Dimensions:    (landcover: 2, model: 14, scenario: 5, era: 4, stream_id: 56460)
Coordinates:
  * landcover  (landcover) float32 8B 0.0 1.0
  * model      (model) float32 56B 0.0 2.0 3.0 4.0 5.0 ... 9.0 12.0 8.0 13.0 1.0
  * scenario   (scenario) float32 20B 0.0 1.0 2.0 3.0 4.0
  * era        (era) float32 16B 0.0 1.0 2.0 3.0
  * stream_id  (stream_id) int32 226kB 1 2 3 4 5 ... 56457 56458 56459 56460
Data variables: (12/52)
    dh1        (landcover, model, scenario, era, stream_id) float32 126MB ...
    dh2        (landcover, model, scenario, era, stream_id) float32 126MB ...
    dh3        (landcover, model, scenario, era, stream_id) float32 126MB ...
    dh4        (landcover, model, scenario, era, stream_id) float32 126MB ...
    dh5        (landcover, model, scenario, era, stream_id) float32 126MB ...
    dh15       (landcover, model, scenario, era, stream_id) float32 126MB ...
    ...         ...
    ra3        (landcover, model, scenario, era, stream_id) float32 126MB ...
    ra8        (landcover, model, scenario, era, stream_id) float32 126MB ...
    spr_ord    (landcover, model, scenario, era, stream_id) float32 126MB ...
    sum_ord    (landcover, model, scenario, era, stream_id) float32 126MB ...
    th1        (landcover, model, scenario, era, stream_id) float32 126MB ...
    tl1        (landcover, model, scenario, era, stream_id) float32 126MB ...
Attributes:
    Data Source:         {'Title': 'Model Input and Output for Hydrologic Sim...
    CMIP5 GCM Metadata:  {'ACCESS1-0': {'Modeling Center': 'Commonwealth Scie...

In [8]:
hru

<xarray.Dataset> Size: 13GB
Dimensions:    (landcover: 2, model: 14, scenario: 5, era: 4, stream_id: 109951)
Coordinates:
  * landcover  (landcover) float32 8B 0.0 1.0
  * model      (model) float32 56B 0.0 2.0 3.0 4.0 5.0 ... 9.0 12.0 8.0 13.0 1.0
  * scenario   (scenario) float32 20B 0.0 1.0 2.0 3.0 4.0
  * era        (era) float32 16B 0.0 1.0 2.0 3.0
  * stream_id  (stream_id) int64 880kB 2118 1865 1874 ... 109450 109495 109568
Data variables: (12/52)
    dh1        (landcover, model, scenario, era, stream_id) float32 246MB ...
    dh2        (landcover, model, scenario, era, stream_id) float32 246MB ...
    dh3        (landcover, model, scenario, era, stream_id) float32 246MB ...
    dh4        (landcover, model, scenario, era, stream_id) float32 246MB ...
    dh5        (landcover, model, scenario, era, stream_id) float32 246MB ...
    dh15       (landcover, model, scenario, era, stream_id) float32 246MB ...
    ...         ...
    ra3        (landcover, model, scenario, era, stream_id) float32 246MB ...
    ra8        (landcover, model, scenario, era, stream_id) float32 246MB ...
    spr_ord    (landcover, model, scenario, era, stream_id) float32 246MB ...
    sum_ord    (landcover, model, scenario, era, stream_id) float32 246MB ...
    th1        (landcover, model, scenario, era, stream_id) float32 246MB ...
    tl1        (landcover, model, scenario, era, stream_id) float32 246MB ...
Attributes:
    Data Source:         {'Title': 'Model Input and Output for Hydrologic Sim...
    CMIP5 GCM Metadata:  {'ACCESS1-0': {'Modeling Center': 'Commonwealth Scie...

In [ ]:
# create a testing function that parses the file name and uses the coords to check the contents of the netCDF
# contents for every statistic are compared to data in the CSV
# this test is only run for stream segments, not HRUs

def test_nc(ds, test_csvs):
    id_col = 'seg_id'
    nc_ids = ds['stream_id'].values
    
    for csv in test_csvs:
        # Read CSV with ID column plus all stats
        df = pd.read_csv(csv, usecols=[id_col] + stats)
        df.replace(-99999, np.nan, inplace=True)
        
        # Filter CSV to only include IDs present in the netCDF
        df = df[df[id_col].isin(nc_ids)].set_index(id_col).reindex(nc_ids).reset_index()

        parts = csv.name.split('_')
        try:
            landcover, model, scenario, era = parts[0], parts[1], parts[2], "_".join([parts[5], parts[6].split(".")[0]])
        except:
            print(f"Error parsing file: {csv.name}")
            continue

        # parse each part of the filename into integers using the encodings_lookup dict
        landcover_int = encodings_lookup['landcover'].get(landcover, landcover)
        model_int = encodings_lookup['model'].get(model, model)
        scenario_int = encodings_lookup['scenario'].get(scenario, scenario)
        era_int = encodings_lookup['era'].get(era, era)

        for stat in stats:
            sel_dict = {"landcover": landcover_int, "model": model_int, "scenario": scenario_int, "era": era_int}
            values = ds[stat].sel(sel_dict).load().values
            csv_values = df[stat].values
            
            assert np.allclose(values, csv_values, equal_nan=True), f"Error in dataset: values for {stat} do not match value in {csv.name}"
            break


In [23]:
# for each geometry, pick 30 random files (~10%) and run the tests
# a successful test will produce no output

seg_test_files = random.sample(seg_files, 20)
test_nc(seg, seg_test_files)

### Important notes about eras / scenarios
These netCDFs have a structure that is partially empty (ie, includes NaNs).

This is due to the fact that all the models each have their own separate modeled historical data values, and do not have a shared historical baseline. Additionally, the projected scenarios do not have modeled historical data. This creates a situation where we have certain dimensional combinations that come up empty. For instance, you could query for the `historical` era of the `rcp60` scenario, but that would just return NaN.

Also, two of the models `ACCESS1-0` and `BNU-ESM` do not have data for scenarios `rcp26` or `rcp60`. So if you were to query for those scenarios, you'd simply get NaN values in return.

See some examples below:

In [27]:
# a query that returns actual values: an emissions scenario and future era

landcover_int = encodings_lookup['landcover'].get("dynamic", None)
model_int = encodings_lookup['model'].get("GFDL-ESM2M", None)
scenario_int = encodings_lookup['scenario'].get("rcp60", None)
era_int = encodings_lookup['era'].get("2046_2075", None)

sel_dict = {"landcover": landcover_int, "model": model_int, "scenario": scenario_int, "era": era_int}

seg["dh3"].sel(
    sel_dict
    ).load().values

array([1.0037e+03, 1.8972e+03, 5.8530e+03, ..., 7.2966e+03, 3.0237e+04,
       2.5204e+01], dtype=float32)

In [28]:
# a query that returns NaN: a historical scenario and future era (combo doesn't really make sense!)

landcover_int = encodings_lookup['landcover'].get("dynamic", None)
model_int = encodings_lookup['model'].get("GFDL-ESM2M", None)
scenario_int = encodings_lookup['scenario'].get("historical", None)
era_int = encodings_lookup['era'].get("2046_2075", None)

sel_dict = {"landcover": landcover_int, "model": model_int, "scenario": scenario_int, "era": era_int}

seg["dh3"].sel(
    sel_dict).load().values

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [29]:
# a query that returns NaN: an emissions scenario and historical era (combo doesn't really make sense!)

landcover_int = encodings_lookup['landcover'].get("dynamic", None)
model_int = encodings_lookup['model'].get("GFDL-ESM2M", None)
scenario_int = encodings_lookup['scenario'].get("rcp60", None)
era_int = encodings_lookup['era'].get("1976_2005", None)

sel_dict = {"landcover": landcover_int, "model": model_int, "scenario": scenario_int, "era": era_int}

seg["dh3"].sel(
    sel_dict
    ).load().values

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [31]:
# a query that returns actual values: a historical scenario and historical era

landcover_int = encodings_lookup['landcover'].get("dynamic", None)
model_int = encodings_lookup['model'].get("GFDL-ESM2M", None)
scenario_int = encodings_lookup['scenario'].get("historical", None)
era_int = encodings_lookup['era'].get("1976_2005", None)   

sel_dict = {"landcover": landcover_int, "model": model_int, "scenario": scenario_int, "era": era_int}

seg["dh3"].sel(
    sel_dict
    ).load().values

array([1.1206e+03, 2.1581e+03, 6.4318e+03, ..., 5.2840e+03, 2.2606e+04,
       9.9587e+00], dtype=float32)